### Import Library

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 3.7 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 448 kB 53.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
import requests
from bs4 import BeautifulSoup
import time

import re

from konlpy.tag import Okt

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import torch
import torchtext
from torchtext.legacy.data import Field, BucketIterator
from torchtext.legacy.data import TabularDataset
from torchtext.vocab import Vectors

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

### Crawling

In [4]:
result = pd.DataFrame()

for i in range(598000, 603255):
    URL = "http://www1.president.go.kr/petitions/" + str(i)

    response = requests.get(URL)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.find('h3', class_='petitionsView_title')
    count = soup.find('span', class_='counter')

    for content in soup.select('div.petitionsView_write > div.View_write'):
        content
    
    a = []
    for tag in soup.select('ul.petitionsView_info_list > li'):
        a.append(tag.contents[1])

    if len(a) != 0:
        df1 = pd.DataFrame({'start':[a[1]],
                            'end':[a[2]],
                            'category':[a[0]],
                            'count':[count.text],
                            'title':[title.text],
                            'content':[content.text.strip()[0:13000]]
                            })
        result = pd.concat([result, df1])
        result.index = np.arange(len(result))

    if i % 60 == 0:
        print('Sleep 90 seconds. Count:' + str(i)
                + ', Local Time:'
                + time.strftime('%Y-%m-%d', time.localtime(time.time()))
                + '' + time.strftime('%X', time.localtime(time.time()))
                + ', Data Length:' + str(len(result)))
        time.sleep(90)

Sleep 90 seconds. Count:598020, Local Time:2021-12-2501:13:57, Data Length:21
Sleep 90 seconds. Count:598080, Local Time:2021-12-2501:15:46, Data Length:81
Sleep 90 seconds. Count:598140, Local Time:2021-12-2501:17:27, Data Length:141
Sleep 90 seconds. Count:598200, Local Time:2021-12-2501:19:09, Data Length:201
Sleep 90 seconds. Count:598260, Local Time:2021-12-2501:20:50, Data Length:261
Sleep 90 seconds. Count:598320, Local Time:2021-12-2501:22:31, Data Length:320
Sleep 90 seconds. Count:598380, Local Time:2021-12-2501:24:11, Data Length:380
Sleep 90 seconds. Count:598440, Local Time:2021-12-2501:25:52, Data Length:440
Sleep 90 seconds. Count:598500, Local Time:2021-12-2501:27:34, Data Length:499
Sleep 90 seconds. Count:598560, Local Time:2021-12-2501:29:09, Data Length:499
Sleep 90 seconds. Count:598620, Local Time:2021-12-2501:30:48, Data Length:537
Sleep 90 seconds. Count:598680, Local Time:2021-12-2501:32:25, Data Length:565
Sleep 90 seconds. Count:598740, Local Time:2021-12-250

In [5]:
print(result.shape)
df = result
df.head()

(2867, 6)


,start,end,category,count,title,content
0,2021-04-28,2021-05-28,기타,"1,812",촉법소년법의 개정을 요구합니다.,안녕하세요? 서울**초등학교의 6학년 1반 학생들입니다. 저희는 아주 불공정한 내용...
1,2021-04-29,2021-05-29,보건복지,801,국산 코로나 백신을 역수입 할 수 있습니다,강원에도 소재하고 있는 ******* 라는 바이오회사 있습니다\n\r\n******...
2,2021-04-29,2021-05-29,기타,"1,794",삼성 이재용 부회장 가석방을 진심으로 청원합니다,삼성 이재용 부회장의 가석방을 진심으로 청원합니다. \n\r\n저는 전남 해남에서 ...
3,2021-04-29,2021-05-29,외교/통일/국방,585,국군장병들은 화이자 백신 맞게 해주세요.,저는 아들과 딸 모두 둔 주부입니다. 지금 저희 아들은 지금 공군에 입대해 있고 딸...
4,2021-04-29,2021-05-29,안전/환경,331,"보이스피싱 취업사기로 피의자가 되고, 보이스피싱 사기대출문자로 피해자가 되었습니다.",보이스피싱 대출문자 차단과 모집책 광고중지 청원\n\r\n[문재인 대통령은 2020...


In [6]:
df.loc[1]['content']

'강원에도 소재하고 있는 ******* 라는 바이오회사 있습니다\n\r\n*******에서 개발 및 임상 중인 코로나 백신은 외국 제품과 달리 무엇보다 안정성이 우수하고 코로나 예방효과가 우수하여\n\r\n현재 필리핀과 기술이전 및 현지 생산시설 건설과 4천만도즈 선 계약에 관한 의향서를 주고 받았습니다\n\r\n국내에서 1, 2 임상 동시 신청 후 1상 완료되었으며 2상 자원자 모집을 하고 있습니다\n\r\n필리핀에서 긴급사용승인이 나면 우리나라에서 역수입 하는 상황이 발생할 수 있습니다\n\r\n외국산 백신에 의존만 하지 마시고\n\r\n아직 임상 진행 중이지만 전문가들 의견을 종합하면 부작용이 거의 없으며 백신 예방 효과가 우수하다는 의견이 대다수 차지 합니다\n\r\n하여 국내에서 개발중인 순수 토종 백신을 해외에서 역수입 하기 전에 긴급사용승인 시 국민의 안전한 백신접종을 위해 *******와 조건부 선구매 예약이라도 하시길 고언 드립니다\n\r\n필리핀에서 긴급사욘승인 나면 역수입 하는 웃픈 상황이 발생하게 됩니다\n\r\n안정성이 우수하고 백신 예방효과 또한 우수한 *******에서 개발 및 임상 중인 백신을 확보해주실것을 청원 합니다\n\n [본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다]'

### Preprocessing

In [7]:
def remove_white_space(text):
    text = re.sub(r'[\t\r\b\f\v]', ' ', str(text))
    return text

In [8]:
def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9]+', ' ', str(text))
    return text

In [9]:
df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

In [10]:
df.loc[1]['content']

'강원에도 소재하고 있는   라는 바이오회사 있습니다   에서 개발 및 임상 중인 코로나 백신은 외국 제품과 달리 무엇보다 안정성이 우수하고 코로나 예방효과가 우수하여   현재 필리핀과 기술이전 및 현지 생산시설 건설과 4천만도즈 선 계약에 관한 의향서를 주고 받았습니다   국내에서 1  2 임상 동시 신청 후 1상 완료되었으며 2상 자원자 모집을 하고 있습니다   필리핀에서 긴급사용승인이 나면 우리나라에서 역수입 하는 상황이 발생할 수 있습니다   외국산 백신에 의존만 하지 마시고   아직 임상 진행 중이지만 전문가들 의견을 종합하면 부작용이 거의 없으며 백신 예방 효과가 우수하다는 의견이 대다수 차지 합니다   하여 국내에서 개발중인 순수 토종 백신을 해외에서 역수입 하기 전에 긴급사용승인 시 국민의 안전한 백신접종을 위해  와 조건부 선구매 예약이라도 하시길 고언 드립니다   필리핀에서 긴급사욘승인 나면 역수입 하는 웃픈 상황이 발생하게 됩니다   안정성이 우수하고 백신 예방효과 또한 우수한  에서 개발 및 임상 중인 백신을 확보해주실것을 청원 합니다   본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다 '

### Tokenizing

In [11]:
okt = Okt()

In [12]:
df['title_token'] = df.title.apply(okt.morphs)

df['content_token'] = df.content.apply(okt.nouns)

### Making Derived Variable

In [13]:
df['token_final'] = df.title_token + df.content_token

df['count'] = df['count'].replace({',' : ''}, regex=True).apply(lambda x : int(x))

df['label'] = df['count'].apply(lambda x : 'Yes' if x >= 1000 else 'No')

print(df.dtypes)

start            object
end              object
category         object
count             int64
title            object
content          object
title_token      object
content_token    object
token_final      object
label            object
dtype: object


In [14]:
df_drop = df[['token_final', 'label']]

### Embedding

In [16]:
embedding_model = Word2Vec(df_drop['token_final'],
                           sg=1,
                           size=100,
                           window=2,
                           min_count=1,
                           workers=4
                           )

print(embedding_model)

Word2Vec(vocab=26088, size=100, alpha=0.025)


In [17]:
model_result = embedding_model.wv.most_similar('음주운전')

print(model_result)

[('뺑소니', 0.9133089184761047), ('형량', 0.9093856811523438), ('미수', 0.9004029631614685), ('살인', 0.8887459635734558), ('범', 0.8882108926773071), ('자살', 0.8842869400978088), ('가정폭력', 0.883514404296875), ('살인자', 0.8816543221473694), ('검거', 0.8769281506538391), ('죄질', 0.8725828528404236)]


In [18]:
embedding_model.wv.save_word2vec_format('/content/drive/MyDrive/Data/nlp_petitions/petitions_tokens_w2v')

loaded_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Data/nlp_petitions/petitions_tokens_w2v')

model_result = loaded_model.most_similar('음주운전')

print(model_result)

[('뺑소니', 0.9133089184761047), ('형량', 0.9093856811523438), ('미수', 0.9004029631614685), ('살인', 0.8887459635734558), ('범', 0.8882108926773071), ('자살', 0.8842869400978088), ('가정폭력', 0.883514404296875), ('살인자', 0.8816543221473694), ('검거', 0.8769281506538391), ('죄질', 0.8725828528404236)]


### Data Split

In [19]:
rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('/content/drive/MyDrive/Data/nlp_petitions/train.csv', index=False, encoding='utf-8-sig')
val.to_csv('/content/drive/MyDrive/Data/nlp_petitions/validation.csv', index=False, encoding='utf-8-sig')

### Define Field Class

In [42]:
def tokenizer(text):
    text = re.sub('[\[\]\']', '', str(text))
    text = text.split(', ')
    return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential=False)

### Data Load

In [43]:
train, validation = TabularDataset.splits(
    path='/content/drive/MyDrive/Data/nlp_petitions/',
    train='train.csv',
    validation='validation.csv',
    format='csv',
    fields=[('text', TEXT), ('label', LABEL)],
    skip_header=True
)

print('Train:', train[0].text, train[0].label)
print('Validation:', validation[0].text, validation[0].label)

Train: ['국민소환제', '를', '입법', '화하여야', '합니다', '부정부패', '및', '무위도식', '국회의원', '국민', '직접', '파면', '수', '국민소환제', '입법', '여의도', '개혁', '입법', '방기', '국회의원', '특권', '때문', '대한민국', '적폐', '원흉', '따라서', '대한민국', '국민', '부정부패', '및', '무위도식', '국회의원', '직접', '파면', '수', '제도', '적장', '치가', '법불', '아귀', '준수', '대한민국', '구현', '수', '이', '정책', '제안', '대통령', '국민소환제', '도입', '법률', '안', '제정', '행해'] Yes
Validation: ['국산', '코로나', '백신', '을', '역', '수입', '할', '수', '있습니다', '강원', '소재', '바이오', '회사', '개발', '및', '임', '중인', '코로나', '백신', '외국', '제품', '달리', '무엇', '안정', '코로나', '예방', '효과', '현재', '필리핀', '기술이전', '및', '현지', '생산', '시설', '건설', '선', '계약', '관', '의향', '를', '국내', '임', '동시', '신청', '후', '상', '완료', '상', '자원', '모집', '필리핀', '긴급', '사용승인', '우리나라', '역', '수입', '상황', '발생', '수', '외국산', '백신', '의존', '임', '진행', '중이', '전문가', '의견', '종합', '부작용', '거의', '백신', '예방', '효과', '의견', '대다수', '차지', '국내', '개발', '순수', '토종', '백신', '해외', '역', '수입', '전', '긴급', '사용승인', '시', '국민', '접종', '위해', '조건', '부', '선', '구매', '예약', '필리핀', '긴급', '욘', '승인', '역', '수입', '웃픈', '상황', '발생

### Define DataLoader

In [44]:
vectors = Vectors(name='/content/drive/MyDrive/Data/nlp_petitions/petitions_tokens_w2v')

TEXT.build_vocab(train, vectors=vectors, min_freq=1, max_size=None)
LABEL.build_vocab(train)

vocab = TEXT.vocab

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [46]:
train_iter, validation_iter = BucketIterator.splits(
    datasets=(train, validation),
    batch_size=8,
    device=device,
    sort=False
)

print('임베딩 벡터의 개수와 차원 : {}'.format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원 : torch.Size([23770, 100])


### TextCNN Modeling

In [47]:
class TextCNN(nn.Module): 
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        
        super(TextCNN, self).__init__()
        
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)      
    
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()                
        self.dropout = nn.Dropout(0.4)         
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)     
        
    def forward(self, x):  
      
        emb_x = self.embed(x)           
        emb_x = emb_x.unsqueeze(1)  

        con_x = [self.relu(conv(emb_x)) for conv in self.convs]       

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]    
        
        fc_x = torch.cat(pool_x, dim=1) 
        fc_x = fc_x.squeeze(-1)       
        fc_x = self.dropout(fc_x)         

        logit = self.fc(fc_x)     
        
        return logit

### Model Function

In [48]:
def train(model, device, train_itr, optimizer):
    
    model.train()                               
    corrects, train_loss = 0.0,0        
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label      
        text = torch.transpose(text, 0, 1)          
        target.data.sub_(1)                                 
        text, target = text.to(device), target.to(device)  

        optimizer.zero_grad()                           
        logit = model(text)                         
    
        loss = F.cross_entropy(logit, target)   
        loss.backward()  
        optimizer.step()  
        
        train_loss += loss.item()    
        result = torch.max(logit,1)[1] 
        corrects += (result.view(target.size()).data == target.data).sum()
        
    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)

    return train_loss, accuracy

### Evaluate Function

In [49]:
def evaluate(model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0, 0

    for batch in itr:
        
        text = batch.text
        target = batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)

        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    test_loss /= len(itr.dataset) 
    accuracy = 100.0 * corrects / len(itr.dataset)
    
    return test_loss, accuracy

### Model Training

In [50]:
model = TextCNN(vocab, 100, 10, [3, 4, 5], 2).to(device)
print(model)

TextCNN(
  (embed): Embedding(23770, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(5, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)


In [51]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_acc =- 1

In [53]:
for epoch in range(1, 10+1):
    tr_loss, tr_acc = train(model, device, train_iter, optimizer)
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))

    val_loss, val_acc = evaluate(model, device, validation_iter)
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))

    if val_acc > best_test_acc:
        best_test_acc = val_acc

        print('model saves at {} accuracy'.format(best_test_acc))
        torch.save(model.state_dict(), 'TextCNN_Best_Validation')

    print('---------------------------------------------------------------------------------------------')

Train Epoch: 1 	 Loss: 0.06264056514789462 	 Accuracy: 76.76547241210938%
Valid Epoch: 1 	 Loss: 0.07610681840262488 	 Accuracy: 67.19023132324219%
---------------------------------------------------------------------------------------------
Train Epoch: 2 	 Loss: 0.04885937886697475 	 Accuracy: 83.3914566040039%
Valid Epoch: 2 	 Loss: 0.07545572207236165 	 Accuracy: 71.20418548583984%
---------------------------------------------------------------------------------------------
Train Epoch: 3 	 Loss: 0.036121083465717414 	 Accuracy: 89.668701171875%
Valid Epoch: 3 	 Loss: 0.077138613395874 	 Accuracy: 71.02967071533203%
---------------------------------------------------------------------------------------------
Train Epoch: 4 	 Loss: 0.024637445700277736 	 Accuracy: 93.59197998046875%
Valid Epoch: 4 	 Loss: 0.08317634879398512 	 Accuracy: 67.71379089355469%
---------------------------------------------------------------------------------------------
Train Epoch: 5 	 Loss: 0.0163885079